<img src="fondo.jpg">

# Quasar Alpha

### Importaciones Lanchain

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from openai import OpenAI
from langchain_openai import ChatOpenAI

#### Audio

In [ ]:
from IPython.display import Audio, display

### Llave OpenAI

In [ ]:
client = OpenAI(api_key='')

## Prompt para el programa

In [ ]:
provider= "Amazon Web Services (AWS)"
llm_ollama = ChatOllama(temperature=0, model="llama3:8b")
llm_gemma2 = ChatOllama(temperature=0, model="gemma2:latest")
llm_openai = ChatOpenAI(temperature=0, model= "gpt-4o", api_key='')

prompt = ChatPromptTemplate.from_messages([
    ("system",
     """Eres un chatbot, tu nombre es Quasar un especialista en devops, infrestructura en la nube, experto en Infrestructura como codigo con Terraform y un especialista en {provider}, sabes usar todos sus servicios para crear infrestructuras, te gusta guiar paso a paso al usuario que no tiene conocimiento de devops, asi que tu guias y le enseñas las mejores alternativas para que pueda llevar a cabo su proyecto, ademas el usuario mismo te dara la clave publica y privada.
     Cuando el usuario esté satisfecho, genera SOLO el código Terraform necesario, sin comentarios ni explicaciones.
Después de generar el código, crea una carpeta llamada 'terraform_project', 
un archivo 'main.tf' dentro de ella con el código, y ejecuta 'terraform init' y 'terraform plan'.
#Contexto
Quasar es un chatbot super amable y paciente, que pregunta cada detalle y es super explicativo para que asi los usuarios se sientan alegres de poder crear infresctructuras en la nube sin necesidad de saber nada, y tu depues de que el usuario este feliz le vas a proporcionar el codigo completo de terraform sin explicaciones, ni comentarios ni nada que no sea el codigo hasta que funcione y luego de que funcione y te agradezca si debes explicar lo que hiciste
#Notas
* Recuerda ser lo mas amigable, explicativa y detallada posible
* Recuerda que cuando ya vayas a generar el codigo solo da el codigo sin comentarios ni explicaciones solo el codigo 
* Las repuestas de explicaciones o ayudas debe ser en español
* Si el usuario no te pregunta algo acerca de temas que no tiene relacion con los temas que te mencione, respondele con que la tematica esta fuera del contecto
* Haz preguntas para saber como puedes ayudarlo de mejor manera y asi no tener inconvenientes
* No debes mencionarle nada acerca de terraform ya que internamente se creara la infrestructura con la ayuda de terraform
* Recuerda cuando el usuario este contento y le parezca bien la idea unicamente genera el codigo de terraform y nada mas que eso, sin ningun tipo de explicacion, ningun texto, ningun comentario, unicamente da el codigo de terraform listo para usar
* No tienes que usar IAM eso lo voy a poner yo mismo
#Ejemplo de un codigo terraform
* Ejemplo de una aplicacion sencilla de una instancia EC2, el usuario te tiene que dar la clave publica y la clave privada, recuerda cuando des el codigo, da unicamente el codigo sin mensajes ni comentarios, SOLO el codigo de terraform, reitero cuando des el codigo de terraform da solo el codigo nada mas que el codigo como esto, una vez que el usuario te haya dado la clave publica y privada no le digas Genial!, Bien o algun mensaje asi, solo dale el codigo de terraform ya que el usuario copiara todo el mensaje y si hay algo q no sea codigo no funcionara
terraform {{
  required_providers {{
    aws = {{
      source  = "hashicorp/aws"
      version = "~> 5.0"
    }}
  }}
}}
provider "aws" {{
  region     = "us-west-1"
  access_key = "clave_publica"
  secret_key = "clave_privada"
}}
resource "aws_instance" "Ubuntu" {{
  ami           = "ami-09d752e13fa8036d3" 
  instance_type = "t2.micro"
  tags = {{
    Name = "Ubuntu-example"
  }}
}}

"""
     ),
    MessagesPlaceholder(variable_name="messages")
])

## Memoria para el Chatbot

In [ ]:
memory = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in memory:
        memory[session_id] = ChatMessageHistory()
    return memory[session_id]

### Cadena de langchain

In [ ]:
chain = prompt | llm_openai
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

### Sesion de chat

In [ ]:
configuracion = {"configurable": {"session_id": "andres"}}

## Reconocimiento de audio (Multimodalidad)

### Entrada del audio

In [ ]:
#entrada de audio
audio_input = 'C:/Users/juanc/OneDrive/Documentos/Universidad/Documentos Materias/Proyectos/Quasar/quasar_backend/input.mp3'
#reproducir audio
display(Audio(audio_input))

### Transcripcion de audio a texto con Whisper

In [ ]:
audio_file= open("C:/Users/juanc/OneDrive/Documentos/Universidad/Documentos Materias/Proyectos/Quasar/quasar_backend/input.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
prompt_input = transcription.text
print(prompt_input)

## Chatbot Quasar AI

### Primer mensaje

In [ ]:

response = with_message_history.invoke(
            {"messages": [HumanMessage(content=prompt_input)], "provider": provider },
            config=configuracion,
        )
response.content

## De texto a audio con tts de OpenAI (Multimodalidad)

In [ ]:
response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input=response.content,
)

response.stream_to_file("nova_output.mp3")

### Resultado 

In [ ]:
#salida de audio
audio_output = 'C:/Users/juanc/OneDrive/Documentos/Universidad/Documentos Materias/Proyectos/Quasar/quasar_backend/nova_output.mp3'
#reproducir audio
display(Audio(audio_output))

## Continuacion de la conversacion

In [ ]:
response = with_message_history.invoke(
            {"messages": [HumanMessage(content="creo que ubuntu estaria bien, y lo demas haz lo que veas conveniente")], "provider": provider },
            config=configuracion,
        )
response.content

In [ ]:
response2 = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input=response.content,
)

response2.stream_to_file("nova_output2.mp3")
#salida de audio
audio_output2 = 'C:/Users/juanc/OneDrive/Documentos/Universidad/Documentos Materias/Proyectos/Quasar/quasar_backend/nova_output2.mp3'
#reproducir audio
display(Audio(audio_output2))

### Paso de las claves de acceso

In [ ]:
response = with_message_history.invoke(
            {"messages": [HumanMessage(content="la clave publica es: ejemplo_clave y la privada es:ejemplo_clave")], "provider": provider},
            config=configuracion,
        )
response.content

### Limpieza de respuesta

In [ ]:
cleaned_hcl = response.content.replace("```hcl\n", "").replace("```", "")

# Replace \n with actual newlines
formatted_hcl = cleaned_hcl.replace(r"\n", "\n")

print(formatted_hcl)

## Uso y pulida del resultado del LLM 

In [ ]:
import os
import subprocess

### Creacion de la infrestructura en la nube con Amazon Web Services (AWS)

In [ ]:
# Nombre de la carpeta
folder_name = "terraform_project"

# Crear la carpeta
os.makedirs(folder_name, exist_ok=True)

# Ruta del archivo main.tf
file_path = os.path.join(folder_name, "main.tf")

# Escribir el código en el archivo main.tf
with open(file_path, "w") as file:
    file.write(formatted_hcl)

# Cambiar el directorio actual a la carpeta creada
os.chdir(folder_name)

# Ejecutar terraform init y mostrar la salida
process = subprocess.Popen(["terraform", "init"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
for line in process.stdout:
    print(line, end="")
process.wait()

# Ejecutar terraform plan y mostrar la salida
process = subprocess.Popen(["terraform", "plan"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
for line in process.stdout:
    print(line, end="")
process.wait()

### Ejecucion del codigo terraform

In [ ]:
# Ejecutar terraform apply y mostrar la salida
print("\nExecuting terraform apply...\n")
process_apply = subprocess.Popen(["terraform", "apply", "-auto-approve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
for line in process_apply.stdout:
    print(line, end="")
process_apply.wait()

<img src="aws_output.png">

### Eliminacion de la instancia

In [ ]:
print("\nExecuting terraform destroy...\n")
process_destroy = subprocess.Popen(["terraform", "destroy", "-auto-approve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
for line in process_destroy.stdout:
    print(line, end="")
process_destroy.wait()